# Checkpoint 1

Implement here your algorithm to interpolate/extrapolate the activation time field from timings recorded in the 20 electrodes of the mapping catheter.

The algorithm should output the following information:
- a reconstruction of the activation times evaluated at each point of a structured grid on the square (-1.5, 1.5)x(-1.5, 1.5),
- an approximation of the conduction velocity (a clinical biomarker) evaluated at each point of a structured grid on the same square.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as spi

from mpl_toolkits.mplot3d import Axes3D  # Ensure 3D plotting support

In [ ]:
# Load the dataset
CP1data = np.load("CP1data.npz")['arr_0']

## Display one recording

In [ ]:
# Select an example recording for visualization
ind_disp = 12  # Index of the sample to display
x_meas = CP1data[ind_disp][0]  # X-coordinates of electrode measurements
y_meas = CP1data[ind_disp][1]  # Y-coordinates of electrode measurements
t_meas = CP1data[ind_disp][2]  # Activation times at each electrode

In [ ]:
# Scatter plot of activation times at measurement points
plt.figure(1)
plt.scatter(x_meas, y_meas, c=t_meas, vmin=0, vmax=np.max(t_meas))
plt.gca().set_aspect(1)
plt.colorbar(label='Time [s]')
plt.title("Measured Activation Times")
plt.show()

## Signature of the function

def checkpoint1_solution( x, y , t, X , Y ):

return time_pred,vel_pred

In [ ]:
# Create a structured grid for interpolation (1501x1501 points in a square region)
X, Y = np.meshgrid(np.linspace(-1.5, 1.5, 1501), np.linspace(-1.5, 1.5, 1501))

def checkpoint1_solution(x, y, t, X, Y):
    """
    Function to interpolate activation times and compute conduction velocity.
    
    Parameters:
    x, y, t : Measured data points (positions and activation times)
    X, Y    : Grid points where predictions are made
    
    Returns:
    time_pred    : Interpolated activation time field
    vel_pred     : Estimated conduction velocity field
    """
    # Interpolate activation times using radial basis function (thin plate spline)
    interp = spi.RBFInterpolator(list(zip(x, y)), t, smoothing=0.1, kernel='thin_plate_spline', epsilon=1)
    time_pred = interp(np.vstack((X.ravel(), Y.ravel())).T).reshape(X.shape)
    
    # Smoothed version of interpolation to reduce noise
    interp_sm = spi.RBFInterpolator(list(zip(x, y)), t, smoothing=0.5, kernel='thin_plate_spline', epsilon=1)
    time_pred_sm = interp_sm(np.vstack((X.ravel(), Y.ravel())).T).reshape(X.shape)
    
    # Compute gradients (approximating conduction velocity as inverse of gradient magnitude)
    grad_x, grad_y = np.gradient(time_pred, (3.0/1500), axis=(0, 1))
    vel_pred = 1.0 / (np.sqrt(grad_x**2 + grad_y**2) + 1e-4)  # Avoid division by zero
    
    # Compute smoothed conduction velocity
    grad_x_sm, grad_y_sm = np.gradient(time_pred_sm, (3.0/1500), axis=(0, 1))
    vel_pred_sm = 1.0 / (np.sqrt(grad_x_sm**2 + grad_y_sm**2) + 1e-4)
    
    # Weighted combination of original and smoothed conduction velocity
    max_value_A = np.max(vel_pred)
    weights = (vel_pred / max_value_A) ** 1.4 * 0.95  # Apply weighting factor
    weighted_average = (vel_pred_sm * weights) + (vel_pred * (1 - weights))
    
    return time_pred, weighted_average

In [ ]:
# Run the function on the selected data sample
time_pred, vel_pred = checkpoint1_solution(x_meas, y_meas, t_meas, X, Y)

In [ ]:
# Plot interpolated activation times
plt.figure()
plt.contour(X, Y, time_pred, 20)
plt.scatter(x_meas, y_meas, c=t_meas, vmin=0, vmax=np.max(time_pred))
plt.colorbar(label='Time [s]')
plt.gca().set_aspect(1)
plt.title('Activation Time Prediction')
plt.show()

In [ ]:
# 3D Surface Plot for Activation Times
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, time_pred, cmap='viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Time [s]')
ax.set_title('3D Activation Time Prediction')
plt.show()

In [ ]:
# 3D Surface Plot for Conduction Velocity
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, vel_pred, cmap='viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Velocity')
ax.set_title('Conduction Velocity Prediction')
fig.colorbar(surf, ax=ax, fraction=0.03, pad=0.1, label='Velocity')
plt.show()